# Laboratorio 1

**Implementación de Bosque Aleatorio (Random Forest)**

**Objetivo:** Predecir del tipo de vulnerabilidades ("SQLi" o "XSS") utilizando los tokens preprocesados.

---
### Paso 1: Importar bibliotecas y cargar el dataset

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Cargar el dataset (ajusta la ruta si es necesario)
df = pd.read_csv('code_vulnerabilities.csv')

# Mostrar las primeras filas para verificar
df.head()

---
### Paso 2: Preprocesamiento de datos
Convertir los tokens preprocesados (que están como strings de listas) en strings unidos por espacios para vectorizarlos. Se ignora la columna "Location" ya que no parece predictiva (es aleatoria). *La etiqueta objetivo es "Vulnerability Type"*.

In [ ]:
# Convertir 'Preprocessed Tokens' a strings (unir tokens con espacios)
df['tokens_str'] = df['Preprocessed Tokens'].apply(lambda x: ' '.join(eval(x)))

# Características (X) y etiquetas (y)
X = df['tokens_str']
y = df['Vulnerability Type']

# Dividir en train/test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorización con TF-IDF (mejor para texto/tokenos)
vectorizer = TfidfVectorizer(max_features=5000)  # Limita features para eficiencia
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

--- 
### Paso 3: Crear y entrenar el modelo de Random Forest
Se aplica **RandomForestClassifier** con parámetros por defecto para simplicidad. <br> 
Puede **ajustar n_estimators** (número de árboles) o **max_depth** para mejorar.

In [ ]:
# Crear el modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrenar el modelo
model.fit(X_train_vec, y_train)

# Predecir en el set de test
predictions = model.predict(X_test_vec)

# Evaluar el modelo
print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

---
### Paso 4: Guardar el modelo (opcional)
Para reutilizarlo sin reentrenar.

In [ ]:
import joblib

# Guardar modelo y vectorizer
joblib.dump(model, 'random_forest_model.pkl')
joblib.dump(vectorizer, 'tfidf_vectorizer.pkl')

**Explicación de cómo utilizar el modelo para predicciones** <br>
Una vez entrenado (o cargado), puedes predecir con valores de entrada propios (un fragmento de código nuevo) o aleatorios (generados para testing).

In [ ]:
model = joblib.load('random_forest_model.pkl')
vectorizer = joblib.load('tfidf_vectorizer.pkl')

#### Predicción con valores propios

- Prepara un nuevo fragmento de código como string.
- Preprocesa: Convierte a tokens (simulando el preprocesamiento original, pero para simplicidad, únelos como string).
- Vectoriza y predice.

In [ ]:
# Ejemplo de input propio (un string similar a 'Preprocessed Tokens' unido)
new_input = ["cursor.execute('SELECT * FROM users WHERE username = ' + user_input"]  # Lista de tokens o string
new_input_str = ' '.join(new_input)  # Une si es lista

# Vectorizar
new_vec = vectorizer.transform([new_input_str])

# Predecir
prediction = model.predict(new_vec)
print("Predicción:", prediction[0])

In [ ]:
# Ejemplo de input propio (un string similar a 'Preprocessed Tokens' unido)
new_input = ["alert('Hello ' + user_input);"]  # Lista de tokens o string
new_input_str = ' '.join(new_input)  # Une si es lista

# Vectorizar
new_vec = vectorizer.transform([new_input_str])

# Predecir
prediction = model.predict(new_vec)
print("Predicción:", prediction[0])

### Predicción con valores aleatorios
Genera inputs aleatorios basados en patrones del dataset para testing.

In [ ]:
import random

# Generar input aleatorio (simulando SQLi o XSS)
def generate_random_input():
    if random.choice([True, False]):  # SQLi
        base = ["db.execute('SELECT", "*", "FROM", random.choice(["users", "products", "employees"]), "WHERE", random.choice(["id", "username", "email"]), "=", "'", "+", "user_input)"]
    else:  # XSS
        base = [random.choice(["alert('Hello", "document.location =", "window.location.href ="]), "'", "+", "user_input;"]
    return ' '.join(base)

# Generar y predecir
random_input_str = generate_random_input()
random_vec = vectorizer.transform([random_input_str])
prediction = model.predict(random_vec)
print("Input aleatorio:", random_input_str)
print("Predicción:", prediction[0])